## Functions used by Introduction to Marine Debris Detection with Sentinel-2 using Python

<hr>

### Import Libraries

In [3]:
# portable way of using operating system dependent functionality
import os

# reads key-value pairs from a .env file and set them as environment variables
# pip install python-dotenv
from dotenv import load_dotenv 
    
# pip install sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

# extends the datatypes used by pandas to allow spatial operations on geometric types
# pip install geopandas
import geopandas as gpd

# finds all the pathnames matching a specified pattern 
import glob

# reads and writes ZIP files
# pip install zipfile36
from zipfile import ZipFile 

# tools for programming and manipulating the Geospatial Data Abstraction Library
# conda install -c conda-forge gdal
from osgeo import gdal, osr

# fundamental package for array computing
import numpy as np

# pip install rasterio
import rasterio as rio 
from rasterio.mask import mask

### Functions

In [ ]:
def Search_And_Download_S2L2A_From_COAH(COAHCredentialsPath, SensingPeriod, CloudCover, BoundingBoxPath, DownloadFolderPath, ProductIndex=0):
    """
    This function searches for Sentinel-2 Level-2A products from Copernicus Open Access Hub and downloads one of the products.
    Input:  COAHCredentialsPath - Path to .env file containing COAH credentials. String.
            SensingPeriod - Sensing Period (StartDate, EndDate). Tuple of strings ("YYYYMMDD", "YYYYMMDD").
            CloudCover - Percentage of Cloud Cover. Tuple of integers (LowerPercentage, UpperPercentage).
            BoundingBoxPath - Path to geojson bounding box representing area of interest. String.
            DownloadFolderPath - Path to folder where the downloaded S2L2A products will be saved. String.
            ProductIndex - Index of the product to download, default is 0 for product with less cloud cover. Integer.
    Output: Downloaded products saved to DownloadFolderPath.
    """
    
    # Load COAH Credentials
    Basepath = os.path.abspath('')
    EnvPath = os.path.join(Basepath,COAHCredentialsPath)
    EVariables = ["COAH_User", "COAH_Password"]
    load_dotenv(EnvPath)
    
    User = os.getenv(EVariables[0])
    Password = os.getenv(EVariables[1])
    if (User != None) and (Password != None):
        print("Credentials successfully loaded.\n")
        
        # Connect using API
        print("Connecting with API...")
        API = SentinelAPI(User, Password, "https://apihub.copernicus.eu/apihub/") # This url may change in the future
        print("Connection established.\n")
    
        # Import Bounding Box
        print("Importing bounding box...")
        BBox = geojson_to_wkt(read_geojson(BoundingBoxPath))
        print("Bounding box successfully imported.\n")
    
        # Search Products
        print("Searching for products...")
        Products = API.query(BBox, date=SensingPeriod, platformname="Sentinel-2", producttype=("S2MSI2A"), cloudcoverpercentage=CloudCover)
        NumberProducts = len(Products.keys())
        print("There are a total of: " + str(NumberProducts) + " products.\n")
        
        if NumberProducts != 0:
        
            # Check Products to Download
            ProductsDF = API.to_geodataframe(Products)
            ProductsDF_Sorted = ProductsDF.sort_values(['cloudcoverpercentage'], ascending=True)
            print("Products sorted by cloud cover percentage: ")
            for i in range(0,NumberProducts):
                print(str(ProductsDF_Sorted.index[i]) + " " + str(ProductsDF_Sorted['cloudcoverpercentage'][i]))
        
            # Create folder (if doesnt exist) to store downloaded product
            if not os.path.exists(DownloadFolderPath):
                os.mkdir(DownloadFolderPath)
        
            # Download Product with less cloud cover or according to index
            print("\nDownloading product: " + str(ProductsDF_Sorted.index[ProductIndex]))
            try:
                API.download(ProductsDF_Sorted.index[ProductIndex], directory_path=DownloadFolderPath)
            except Exception as e:
                print(str(e))
            print("Done.")
           
    else:
        print("Unable to load credentials.\n")
    

In [ ]:
def Unzip_Latest(ZipFolderPath):
    """
    This function reads the latest zip file from a folder and unzips it to the same folder.
    Input:  ZipFolderPath - Path to folder containing zip files. String.
    Output: LatestSafeProduct - Path to latest SAFE product. String.
    """
    
    # Select the latest downloaded zip product.
    # Note: If an old zip product is modified (e.g. name), it will become the latest one.
    ListOfZipProducts = glob.glob(os.path.join(ZipFolderPath,"*.zip"))
    LatestZipProduct = max(ListOfZipProducts, key=os.path.getctime)
    print("Unzipping: " + str(os.path.basename(LatestZipProduct)))

    # Unzip it to SAFE
    ZipFile(LatestZipProduct,"r").extractall(ZipFolderPath)

    # Return path of SAFE file (recently unzipped)
    ListOfSafeProducts = glob.glob(os.path.join(ZipFolderPath,"*.SAFE"))
    LatestSafeProduct = max(ListOfSafeProducts, key=os.path.getctime)

    return LatestSafeProduct 

In [ ]:
def Select_Bands_Of_Interest(SAFEProductPath):
    """
    This function provides a list with paths to the bands of interest - B04, B06, B08, B11, SLC and TCI.
    Input:  SAFEProductPath - Path to S2L2A SAFE product. String.
    Output: BandsOfInterestList - List with paths to the bands of interest. List of strings.
    """
    
    # Path to IMG_DATA
    GRANULEPath = os.path.join(SAFEProductPath, "GRANULE")
    IMG_DATAPath = os.path.join(glob.glob(GRANULEPath + "/*")[0], "IMG_DATA")

    # Bands of interest
    B04_10m = glob.glob(os.path.join(IMG_DATAPath, "R10m", "*B04_10m.jp2"))[0]
    B06_20m = glob.glob(os.path.join(IMG_DATAPath, "R20m", "*B06_20m.jp2"))[0]
    B08_10m = glob.glob(os.path.join(IMG_DATAPath, "R10m", "*B08_10m.jp2"))[0]
    B11_20m = glob.glob(os.path.join(IMG_DATAPath, "R20m", "*B11_20m.jp2"))[0]
    SCL_20m = glob.glob(os.path.join(IMG_DATAPath, "R20m", "*SCL_20m.jp2"))[0]
    TCI_10m = glob.glob(os.path.join(IMG_DATAPath, "R10m", "*TCI_10m.jp2"))[0]

    BandsOfInterestList = [B04_10m, B06_20m, B08_10m, B11_20m, SCL_20m, TCI_10m]
    
    return BandsOfInterestList

In [ ]:
def Raster_MainInfo(BandPath):
    """
    This function prints the main information about a band raster and outputs the projection used by the product.
    Input: BandPath - Path to the band. String.
    Output: EPSGcode - EPSG code of the projection. Integer. 
    """
    
    # Read band raster
    Raster = gdal.Open(BandPath)

    #Get projection EPSG 
    RasterProjection = osr.SpatialReference(wkt=Raster.GetProjection())
    RasterEPSG = str(RasterProjection.GetAttrValue('AUTHORITY',1))
    print("Projection EPSG: " + RasterEPSG)
    EPSGcode = int(RasterEPSG)

    # Get some information and print Spatial Resolution
    ulx, xres, xskew, uly, yskew, yres  = Raster.GetGeoTransform()
    print("Spatial Resolution (X x Y): " + str(xres) + " x " + str(yres))

    # Calculate bounds and print them
    lrx = ulx + (Raster.RasterXSize * xres)
    lry = uly + (Raster.RasterYSize * yres)
    print("Bounds [ulx, uly, lrx, lry]: " + str([ulx, uly, lrx, lry]))

    # Print raster size
    print("Size (X x Y): " + str(Raster.RasterXSize) + " x " + str(Raster.RasterYSize))
    
    return EPSGcode

In [ ]:
def Apply_ScaleFactor_and_Resample(BandPath, SavingFolderPath):
    """
    This function applies a scale factor of 10000 to bands B04, B06, B08 and B11. A resampling is done using the 
    bicubic method, except for SCL that uses a nearest neighbour method.  
    Input: BandPath - Path to the band. String.
           SavingFolderPath - Path to folder where the band will be saved. String.
    Output: Final band is saved as tiff to a folder.
    """
    
    # Read band
    BandName = os.path.basename(BandPath)[-11:-8]
    print("Scalling, resampling and saving: " + BandName)
    Raster = gdal.Open(BandPath)
    Band = Raster.GetRasterBand(1)
    Data = Band.ReadAsArray()

    # Change resampling method and scale factor according to band
    if BandName != "SCL":
        ResampleAlgorithm = gdal.GRA_Cubic
        ScaleFactor = 10000
    else:
        ResampleAlgorithm = gdal.GRA_NearestNeighbour
        ScaleFactor = 1

    # Apply scale factor
    DataAfterScale = Data/ScaleFactor

    # Save the scalled data temporarily as float
    Driver = gdal.GetDriverByName("GTiff")
    RasterAfterScale = Driver.Create("/vsimem/BandAfterScale.tiff", Raster.RasterXSize, Raster.RasterYSize, 1, gdal.GDT_Float32)
    RasterAfterScale.SetProjection(Raster.GetProjectionRef())
    RasterAfterScale.SetGeoTransform(Raster.GetGeoTransform()) 
    RasterAfterScaleBand = RasterAfterScale.GetRasterBand(1)
    RasterAfterScaleBand.WriteArray(DataAfterScale)

    # Resample the data, save to a temporary folder. GDAL Warp ignores bands already with 10m.
    RasterAfterResampling = gdal.Warp("/vsimem/BandAfterResampling.tiff", RasterAfterScale, xRes=10, yRes=10, resampleAlg = ResampleAlgorithm, dstSRS = "EPSG:"+str(EPSGcode), dstNodata=0)
        
    # GDAL warp creates negative values during resampling, they need to be converted to 0 (no data)
    RasterAfterResamplingBand = RasterAfterResampling.GetRasterBand(1)
    DataAfterResampling = RasterAfterResamplingBand.ReadAsArray()
    FinalData = np.where(DataAfterResampling<0, 0, DataAfterResampling)
    
    # Save the final bands to the hidden folder
    BandNameTiff = BandName + ".tiff"
    FinalRaster = Driver.Create(os.path.join(SavingFolderPath,BandNameTiff), RasterAfterResampling.RasterXSize, RasterAfterResampling.RasterYSize, 1, gdal.GDT_Float32)
    FinalRaster.SetProjection(RasterAfterResampling.GetProjectionRef())
    FinalRaster.SetGeoTransform(RasterAfterResampling.GetGeoTransform()) 
    FinalRasterBand = FinalRaster.GetRasterBand(1)
    FinalRasterBand.WriteArray(FinalData)
    FinalRaster = None

In [ ]:
def Feature_Of_Interest_From_Shapefile(ShapefilePath, FeatureQuerry, EPSGcode, PlotFeature):
    """
    This function reads a shapefile and selects the feature of interest.
    Input:  ShapefilePath - Path to Shapefile (.shp). String.
            FeatureQuerry - Feature of interest from shapefile. E.g.: "continent == 'Africa'". String.
                            Check attribute table of shapefile.
            EPSGcode - Reproject using the desired EPSG code. Integer.  
            PlotFeature - Basic plot of the feature. True or False as Bool.
    Output: FeatureObject - Feature of interest.
    """
    
    # Read shapefile
    ShapefileObject = gpd.read_file(ShapefilePath)
    
    # Select feature
    FeatureObject = ShapefileObject.query(FeatureQuerry)

    # Reproject feature
    print("Shapefile feature original projection: " + str(FeatureObject.crs))
    FeatureObject = FeatureObject.to_crs(EPSGcode)
    print("Shapefile feature projection after reprojection: " + str(FeatureObject.crs))
    
    # Plot feature
    if PlotFeature == True:
        FeatureObject.plot()

    return FeatureObject

In [ ]:
def Apply_LandMask_to_Band(BandPath, LandMask, SavingFolderPath):
    """
    This function masks out land from a band and saves it replacing the old band.  
    Input: BandPath - Path to the band. String.
           LandMask - Shapefile feature. Object.
           SavingFolderPath - Path to folder where the masked band will be saved. String.
    Output: Masked band is saved as tiff to a folder, replacing the old one.
    """

    # Apply land mask to band
    with rio.open(BandPath) as Raster:
        MaskedRasterData, MaskedRasterTransform = mask(Raster, LandMask.geometry, invert=True)
        MaskedRasterMeta = Raster.meta.copy()
        MaskedRasterMeta.update({"driver": "GTiff", "height": MaskedRasterData.shape[1], "width": MaskedRasterData.shape[2], "transform": MaskedRasterTransform})
                        
    # Save new band replacing the old one    
    with rio.open(BandPath, "w", **MaskedRasterMeta) as MaskedRaster:
        MaskedRaster.write(MaskedRasterData)

In [ ]:
def Create_and_Apply_CloudMask_to_Band(BandPath, SCLBandPath, SavingFolderPath):
    """
    This function masks out clouds (giving value of 0) from a band using the scene classification map 
    and saves it replacing the old band.
    The following labels from the SCL are used:
    - CLOUD_SHADOW
    - CLOUD_MEDIUM_PROBA
    - CLOUD_HIGH_PROBA
    - THIN_CIRRUS
    Input: BandPath - Path to the band. String.
           SCLBandPath - Path to the SCL where the cloud mask will be extracted. String.
           SavingFolderPath - Path to folder where the masked band will be saved. String.
    Output: Masked band is saved as tiff to a folder, replacing the old one.
    """

    # Read band as array
    Raster = gdal.Open(BandPath)
    RasterBand = Raster.GetRasterBand(1)
    RasterData = RasterBand.ReadAsArray()

    # Read SCL as array
    SCLRaster = gdal.Open(SCLBandPath)
    SCLRasterBand = SCLRaster.GetRasterBand(1)
    SCLRasterData = SCLRasterBand.ReadAsArray()
  
    # Apply filter
    # CLOUD_SHADOW: 3
    RasterData[SCLRasterData==3] = 0
    # CLOUD_MEDIUM_PROBA: 8
    RasterData[SCLRasterData==8] = 0
    # CLOUD_HIGH_PROBA: 9
    RasterData[SCLRasterData==9] = 0
    # THIN_CIRRUS: 10
    RasterData[SCLRasterData==10] = 0

    # Save new band replacing the old one
    Driver = gdal.GetDriverByName("GTiff")
    MaskedRaster = Driver.Create(BandPath, Raster.RasterXSize, Raster.RasterYSize, 1, gdal.GDT_Float32)
    MaskedRaster.SetProjection(Raster.GetProjectionRef())
    MaskedRaster.SetGeoTransform(Raster.GetGeoTransform()) 
    MaskedRasterBand = MaskedRaster.GetRasterBand(1)
    MaskedRasterBand.WriteArray(RasterData)
    MaskedRaster = None

In [ ]:
def Calculate_FAI_or_FDI(S2SAFEProductPath, BandsFolder, SavingFolder, Index):
    """
    This function calculates the Floating Algae Index or Floating Debris Index and saves to a folder.
    Input: S2SAFEProductPath - Path to SAFE product, important to distinguish between S2A or S2B. String.
           BandsFolder - Path to folder containing bands. String.
           SavingFolder - Path to folder where the index will be saved. String.
           Index - FAI or FDI. String.
    Output: IndexPath - Path to the index (FAI or FDI) tiff saved on folder.
    """
    
    # Select central wavelength according to Sentinel-2 platform (S2-A or -B)
    if os.path.basename(S2SAFEProductPath)[0:3] == "S2A":
        wlB04 = 664.6
        wlB08 = 832.8
        wlB11 = 1613.7
    else:
        wlB04 = 664.9
        wlB08 = 832.9
        wlB11 = 1610.4
    
    # Open bands as arrays
    B04Raster = gdal.Open(os.path.join(BandsFolder, "B04.tiff"))
    B04Band = B04Raster.GetRasterBand(1)
    B04Data = B04Band.ReadAsArray()

    B06Raster = gdal.Open(os.path.join(BandsFolder, "B06.tiff"))
    B06Band = B06Raster.GetRasterBand(1)
    B06Data = B06Band.ReadAsArray()

    B08Raster = gdal.Open(os.path.join(BandsFolder, "B08.tiff"))
    B08Band = B08Raster.GetRasterBand(1)
    B08Data = B08Band.ReadAsArray()

    B11Raster = gdal.Open(os.path.join(BandsFolder, "B11.tiff"))
    B11Band = B11Raster.GetRasterBand(1)
    B11Data = B11Band.ReadAsArray()

    # Calculate FAI or FDI
    if Index == "FAI":
        print("Calculating FAI and saving... ")
        BandNameTiff = "FAI.tiff"
        IndexData = B08Data - (B04Data + (B11Data-B04Data)*((wlB08-wlB04)/(wlB11-wlB04)))
    else:
        BandNameTiff = "FDI.tiff"
        print("Calculating FDI and saving... ")
        IndexData = B08Data - (B06Data + (B11Data-B06Data)*((wlB08-wlB04)/(wlB11-wlB04))*10)
    
    # Save Index
    Driver = gdal.GetDriverByName("GTiff")
    IndexPath = os.path.join(SavingFolder,BandNameTiff)
    IndexRaster = Driver.Create(IndexPath, B04Raster.RasterXSize, B04Raster.RasterYSize, 1, gdal.GDT_Float32)
    IndexRaster.SetProjection(B04Raster.GetProjectionRef())
    IndexRaster.SetGeoTransform(B04Raster.GetGeoTransform()) 
    IndexRasterBand = IndexRaster.GetRasterBand(1)
    IndexRasterBand.WriteArray(IndexData)
    IndexRaster = None
    
    print("Done.")
    
    return IndexPath